In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_github = user_secrets.get_secret("github")
! rm -rf feedback2021
! git clone https://{secret_github}@github.com/VilmosProkaj/feedback2021.git
! pip install feedback2021/

Cloning into 'feedback2021'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 216 (delta 105), reused 163 (delta 52), pack-reused 0
Receiving objects: 100% (216/216), 59.29 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Processing ./feedback2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for feedback2021: filename=feedback2021-0.0.post1.dev30+g983bd4b-py3-none-any.whl size=23561 

In [2]:
! pip install -U jax jaxlib dm_haiku optax
#!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# ! pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.
## !pip install --upgrade jax jaxlib==0.3.0+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html
#! pip install dm_haiku

     |████████████████████████████████| 912 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 65.4 MB 101 kB/s  eta 0:00:01
     |████████████████████████████████| 309 kB 71.1 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 68.1 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 6.5 MB/s  eta 0:00:01
  Created wheel for jax: filename=jax-0.3.1-py3-none-any.whl size=1054277 sha256=8231389b0c076a1ad64df0231f73faff2edda1698e613665c853b61367700685
  Stored in directory: /root/.cache/pip/wheels/04/14/e8/ee9de500f173ec900a5167686d9bb17c0171ed678680b96a57
Successfully built jax
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.70
    Uninstalling jaxlib-0.1.70:
      Successfully uninstalled jaxlib-0.1.70
  Attempting uninstall:

In [3]:
import kaggle_init

In [4]:
kaggle_init.on_kaggle(), kaggle_init.is_cuda_available(), kaggle_init.is_tpu_available()

(True, False, True)

In [5]:
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=.75
compute_on_tpu = True
if compute_on_tpu:
    if kaggle_init.is_tpu_available():
        from feedback2021.jax_tpu_init import jax_tpu_init
        jax_tpu_init()       
    else:
        import os
        os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=2'
#import jax
#jax.devices()

In [ ]:
# jax_tpu_init??

In [ ]:
LOG_TO_WANDB = False

In [ ]:
if LOG_TO_WANDB:
    !pip install --upgrade wandb -q # experiment tracking

In [ ]:
if LOG_TO_WANDB:
    import wandb
    import os
    os.environ["WANDB_PROJECT"] = "kaggle_feedback"
    os.environ["WANDB_ENTITY"] = "prvi"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "gradient"

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("wandb")
        os.environ["WANDB_API_KEY"] = api_key
        wandb.login()
        wandb.init(dir="/tmp/") 
    except:
        print('If you want to use your W&B account, '
              'go to Add-ons -> Secrets and provide your W&B access token.\n'
              'Use the Label name `wandb`. \n'
              'Get your W&B access token from here: https://wandb.ai/authorize')

In [6]:
from feedback2021.helper import id2label, label2id

from feedback2021.prepare_data import (create_train_dataset_pd, 
                                       to_chunk_data, 
                                       chunk_mapping,
                                       add_input_ids, 
                                       add_labels,
                                       add_rle_word2,
                                       has_name,
                                    )

import feedback2021.metric as metric

from feedback2021.postprocess import (mk_metric, 
                                      mk_prediction_transform, 
                                      mk_binary_metric, 
                                      mk_binary_prediction_transform)

from feedback2021.visualize import show_result

import feedback2021.hk_roberta as hk_roberta

In [7]:
# CONFIG
just_test = False
experiment_id = 1
task = "token_classification"
model_checkpoint = "roberta-large" 
# "allenai/longformer-base-4096" 
# "distilroberta-base" # "microsoft/deberta-v3-xsmall" #"roberta-base"
if just_test:
    max_length = 128
    stride = 128
else:
    max_length = 512
    stride = 128
min_tokens = 6
model_path = f'{model_checkpoint.split("/")[-1]}-{experiment_id}'
data_from_wandb = False
save_to_wandb = False or not data_from_wandb


## Data Preprocessing

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#import tensorflow.config
#tensorflow.config.experimental.set_visible_devices([], "GPU")

In [9]:
from feedback2021.clean_train_data import mk_clean_train_data
if just_test:
    cleaned_train = mk_clean_train_data(num_records=100) #cleaned_train[:100]
else:
    cleaned_train = mk_clean_train_data()

data = create_train_dataset_pd(cleaned_train_df=cleaned_train, 
                               tokenizer=tokenizer, 
                               verbose=True)

from sklearn.model_selection import train_test_split

data = dict(zip(['train','test'], 
                train_test_split(data,
                                 test_size=0.1, 
                                 shuffle=True, 
                                 random_state=42)))


2022-02-26 04:39:04.191960: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-26 04:39:08.509334: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-26 04:39:08.509403: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


reformat train data:   0%|          | 0/144293 [00:00<?, ?it/s]

reading essays:   0%|          | 0/15594 [00:00<?, ?it/s]

searching for disaligned labels:   0%|          | 0/15594 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors


    word mapping:   0%|          | 0/15594 [00:00<?, ?it/s]

adding token rle:   0%|          | 0/15594 [00:00<?, ?it/s]

 adding label:   0%|          | 0/15594 [00:00<?, ?it/s]

In [ ]:
data['train'].head(), data['test'].head()

In [ ]:
if LOG_TO_WANDB and save_to_wandb:
    data.remove_columns([#'input_ids', 
                         #'rle_token', 
                         'labels', 
                         #'offset_mapping'
    ]).save_to_disk('data')

    artifact = wandb.Artifact('data', description='train test split', type='dataset')
    artifact.add_dir('data')
    wandb.log_artifact(artifact)
    !ls -sRh data
    

In [ ]:
# if  not isinstance(data.column_names, dict):
#     data = data.train_test_split(test_size=0.1, shuffle=True, seed=42)
    
# chunk_data = to_chunk_data(data, 
#                            chunk_len=max_length, 
#                            stride=stride, 
#                            prefix=[tokenizer.bos_token_id],
#                            postfix=[tokenizer.eos_token_id])
# chunk_data

In [10]:
def chop_up(data, tokenizer, max_length, stride):
    examples = {k: list(data[k]) for k in data.columns}
    examples['labels'] = list(data['labels'])
    examples['id'] = list(data.index.values)
    f = chunk_mapping(chunk_len=max_length, 
                      stride=stride, 
                      prefix=[tokenizer.bos_token_id],
                      postfix=[tokenizer.eos_token_id])
    return f(examples)

def to_records(data):
    return [dict(zip(data.keys(), rec))  for rec in zip(*data.values())]
    

In [11]:
chunk_data = {
    k: chop_up(v, tokenizer, max_length=max_length, stride=stride)
    for k,v in data.items()
}

In [12]:
import yaml
print(yaml.dump(
    {
        k: {
            k0: f'size={len(v0[0])}, type={type(v0[0][0]).__name__}' 
            if isinstance(v0[0], list) else v0[0] 
            for k0, v0 in v.items()
        } 
     for k,v in chunk_data.items()
    }
))

test:
  input_ids: size=216, type=int
  labels: size=216, type=int
  offset: 0
  text_id: AA0B42CF00A6
train:
  input_ids: size=512, type=int
  labels: size=512, type=int
  offset: 0
  text_id: 838AA0C9A269



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# if not has_name(data, 'rle_word'):
#     from feedback2021.helper import Block
#     def add_rle_word2(data):
#         # assert has_name(data, 'offset_mapping'), 'add input_ids first!'
#         if not has_name(data,'word_mapping'):
#             data = add_word_mapping(data)
#         return data.map(lambda x: {'rle_word': [Block(t).inv_map(x['word_mapping']) 
#                                                  for t in x['rle_char']]},
#                             desc='rle to word coordinates')
#     data = add_rle_word2(data)

In [ ]:
# from collections import defaultdict
# import numpy as np
# word_counts = defaultdict(list)
# from  datasets import concatenate_datasets
# all_data = concatenate_datasets(list(data.values()))
# for x in all_data['rle_word']:
#     for cls_id, start, end  in x:
#         word_counts[id2label[cls_id]].append(end-start)

# for k, v in word_counts.items():
#     plt.hist(v,bins=np.arange(1,max(v)+1)-0.5)
#     plt.title(f'{k} min:{min(v)} max: {max(v)}, 2%,5%: {np.percentile(v,[2,5])}')
#     plt.show()

## Model and Training

In [ ]:
# chunk_data = chunk_data.rename_column('labels','label')
# DataLoader?

In [ ]:
# import numpy as np
# def mk_collate_fn(tokenizer, max_length=512):
#     def collate_fn(features):
        
#         batch ={key: np.array([f[key]+[t]*(max_length-len(f[key])) for f in features], dtype=np.int32)
#                 for t,key in zip([tokenizer.pad_token_id,-100],['input_ids','labels'])}
#         return batch
#     return collate_fn
# from torch.utils.data import DataLoader

# train_dataset = DataLoader(chunk_data_list, shuffle=True, #.remove_columns(['offset', 'text_id']), 
#                            batch_size=16,
#                            collate_fn=mk_collate_fn(tokenizer, 512))

In [13]:
%%time
import tensorflow.data as tfdata
#ds=
import numpy as np

def mk_collate_fn(tokenizer, max_length=512):
    def collate_fn(features):
        
        batch = features.copy()
        for t, key in zip([tokenizer.pad_token_id,-100], ['input_ids','labels']):
            batch[key] = np.array([f+[t]*(max_length-len(f)) for f in batch[key]], 
                                  dtype=np.int32)
        return batch
    
    return collate_fn

    
collate_fn = mk_collate_fn(tokenizer, max_length)

ds ={k: tfdata.Dataset.from_tensor_slices(collate_fn(v)) for k,v in chunk_data.items()}
#(chunk_data_list)

CPU times: user 12.1 s, sys: 540 ms, total: 12.7 s
Wall time: 12.6 s


In [14]:
batch_size = 8 if kaggle_init.is_tpu_available() else 2
dummy_data = tfdata.Dataset.from_tensor_slices(
    {
        'input_ids': np.zeros((batch_size, max_length), dtype=np.int32),
        'labels': -100*np.ones((batch_size, max_length), dtype=np.int32),
        'text_id': np.array(['0'*11]*batch_size),
        'offset': np.array(np.zeros(batch_size, dtype=np.int32))
    }
)

In [ ]:
dummy_data

In [15]:
import jax
jax.devices(), jax.device_count()

([TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
  TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
  TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
  TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
  TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
  TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
  TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
  TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)],
 8)

In [16]:
def subset(x):
    return {k: x[k] for k in ['input_ids', 'labels', 'offset']}

train_dataset = (ds['train'].
                 map(subset).
                 repeat().
                 shuffle(4096).
                 batch(batch_size=batch_size).
                 batch(batch_size=jax.device_count()).as_numpy_iterator())

test_dataset = (ds['test'].
                concatenate(dummy_data.repeat(jax.device_count())).
                map(subset).batch(batch_size=batch_size, drop_remainder=True).
                batch(batch_size=jax.device_count(), drop_remainder=True))

In [17]:
len(ds['test']), len(ds['train'])

(5116, 43645)

In [ ]:
class Metric:
    def __init__(self):
        self.reset()
    def reset(self):
        self._value = 0
        self._n = 0
    def update(self,v):
        self._n += 1
        self._value += v
    def value(self):
        return self._value/self._n

In [39]:
{k: f'shape={v.shape}, dtype={v.dtype}' for k,v in next(iter(train_dataset)).items()}

{'input_ids': 'shape=(8, 16, 512), dtype=int32',
 'labels': 'shape=(8, 16, 512), dtype=int32',
 'offset': 'shape=(8, 16), dtype=int32'}

In [ ]:
local_device_count = jax.device_count()
local_device_count

In [19]:
import haiku as hk
import jax.numpy as jnp
import optax
import functools
from typing import Any, Mapping

def mk_loss_fn(config):
    
    def loss_fn(logits, data):
    #                params,
    #                rng,
    #                data: Mapping[str, jnp.ndarray],
    #                is_training: bool = True) -> jnp.ndarray:
        """Compute the loss on data wrt params."""


    #     logits = forward_fn(params, rng, data, is_training)
        targets = jax.nn.one_hot(data['labels'], len(config['id2label']))
        assert logits.shape == targets.shape

        mask = jnp.not_equal(data['input_ids'], config['pad_token_id'])
        mask = mask * jnp.greater_equal(data['labels'], 0)
        loss = -jnp.sum(targets * jax.nn.log_softmax(logits), axis=-1)
        loss = jnp.sum(loss * mask) / jnp.clip(jnp.sum(mask),1)

        return loss
    
    return loss_fn

class Updater:
    """A stateless abstraction around an init_fn/update_fn pair.
    This extracts some common boilerplate from the training loop.
    """

    def __init__(self, 
                 net, 
                 loss_fn,
                 optimizer: optax.GradientTransformation):
        
        self._net_init = net.init
        self._loss_fn = lambda params,rng, data: loss_fn(net.apply(params, rng, data, is_training=True), data)
        self._opt = optimizer

    @functools.partial(jax.jit, static_argnums=0)
    def init(self, rng, data, pretrained_params=None):
        """Initializes state of the updater."""
        out_rng, init_rng = jax.random.split(rng)
        params = self._net_init(init_rng, data)
        if pretrained_params is not None:
            params = hk.data_structures.merge(params, pretrained_params)
        #params = hk.data_structures.map(lambda x: jnp.stack([x]*local_device_count), params)
        opt_state = self._opt.init(params)
        # rng = jax.random.PRNGKey(FLAGS.train_init_random_seed)
        #rng = jnp.broadcast_to(rng, (local_device_count,) + rng.shape)
        out = dict(
            step=np.array(0),
            rng=out_rng,
            opt_state=opt_state,
            params=params,
            loss=np.array(0),
        )
        return out

    @functools.partial(jax.jit, static_argnums=0)
    def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
        """Updates the state using some data and returns metrics."""
        rng, new_rng = jax.random.split(state['rng'])
        params = state['params']
        loss, grads = jax.value_and_grad(self._loss_fn)(params, rng, data)

        if compute_on_tpu:
            grads = jax.lax.pmean(grads, 'i')


        updates, opt_state = self._opt.update(grads, state['opt_state'])
        params = optax.apply_updates(params, updates)

        new_state = {
            'step': state['step'] + 1,
            'rng': new_rng,
            'opt_state': opt_state,
            'params': params,
            'loss': state['loss'] + loss
        }

        return new_state 

In [20]:
def get_predictions(net, loss_fn, state, test_ds):

    def run_net(state, data):
        logits = net.apply(state['params'], state['rng'], data, is_training=False)
        loss = loss_fn(logits, data)
        pred = jax.numpy.argmax(logits, axis=-1)
        state['loss'] = state['loss'] + loss
        return pred, state
        
    
    run_net = jax.pmap(run_net, axis_name='i')
    predictions = []
    for batch in test_ds.as_numpy_iterator():
        pred, state = run_net(state, batch)
        predictions.append(pred)
        
    predictions = [ pred 
                    for preds in jax.device_get(predictions) 
                    for pred in preds.reshape((-1,) + preds.shape[2:]) ]
    return predictions, state

def eval_net(net, loss_fn, state, test_ds):

    def run_net(state, data, loss, cnt):
        cnt = cnt + jnp.greater(data['labels'].max(axis=-1),0).sum()
        logits = net.apply(state['params'], state['rng'], data, is_training=False)
        loss = loss + loss_fn(logits, data)
        #pred = jax.numpy.argmax(logits, axis=-1)
        #state['loss'] = state['loss'] + loss
        return loss, cnt
        
    
    run_net = jax.pmap(run_net, axis_name='i')
    #predictions = []
    loss = np.zeros(jax.device_count())
    cnt = np.zeros(jax.device_count())
    for batch in test_ds.as_numpy_iterator():
        loss, cnt = run_net(state, batch, loss, cnt)
        # predictions.append(pred)
    return float(loss.sum()/cnt.sum())

#     predictions = [ pred 
#                     for preds in jax.device_get(predictions) 
#                     for pred in preds.reshape((-1,) + preds.shape[2:]) ]
#     return predictions, state

        

In [21]:
class Log:
    def __init__(self, step=0):
        self.step = step
        self.start_time = self.prev_time = time.time()
        
    def update(self, state):
        step = int(state['step'][0])
        step_delta, self.step = step-self.step, step
        
        c_time = time.time()
        time_delta, self.prev_time = c_time - self.prev_time, c_time
        
        loss = float(state['loss'].mean())/step_delta
        state['loss'] = 0*state['loss']
        
        return {'step': self.step,
                'loss': loss,
                'elapsed_time': time_delta,
                'total_time':   c_time-self.start_time,
                'iter_per_sec': time_delta/max(1,step_delta),
                'sec_per_iter': step_delta/max(1,time_delta),
               }

def format_log(log):
    return "iteration: {step:<5}, loss: {loss:.4f}, elapsed_time:{elapsed_time:.2f}".format(**log)

In [22]:
import logging
import time
from transformers import AutoConfig
from feedback2021.hk_roberta import (build_forward_fn, 
                                     lm_loss_fn, 
                                     # GradientUpdater, 
                                     hk, optax, functools, Mapping, jnp, jax)

## global variables
# batch_size = 16  # Train batch size per core
total_batch_size = batch_size*jax.device_count()
learning_rate = 2.5e-5*(total_batch_size/32) # Max learning-rate
grad_clip_value = 0.15  # Gradient norm clip value

checkpoint_dir = '/jax-transformer'  # Directory to store checkpoints
if just_test:
    LOG_EVERY = 2
    MAX_STEPS = 4
else:
    LOG_EVERY = len(ds['train'])//(2*total_batch_size)
    MAX_STEPS = (3*len(ds['train']))//total_batch_size+1

config = AutoConfig.from_pretrained(model_checkpoint, #'distilroberta-base', 
                                    label2id=label2id, 
                                    id2label=id2label).to_dict()
translation = hk_roberta.weight_name_translation(config=config, prefix='roberta')
pretrained_params = hk_roberta.load_hf_pytorch_weights(model_checkpoint, #'distilroberta-base', 
                                                       translation)

logging.info = print
logging.info('Starting...')

forward_fn = build_forward_fn(config)

forward_fn = hk.transform(forward_fn)

#loss_fn = functools.partial(lm_loss_fn, forward_fn.apply, config)
loss_fn = mk_loss_fn(config)

lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=min(50, MAX_STEPS//6), 
                                               decay_steps=MAX_STEPS-min(50,MAX_STEPS//6), 
                                               end_value=1e-4)

optimizer = optax.chain(
        optax.clip_by_global_norm(grad_clip_value),
        optax.adam(learning_rate, b1=0.9, b2=0.99),
        optax.scale_by_schedule(lr_schedule),
    )

updater = Updater(forward_fn, loss_fn, optimizer)

# Initialize parameters.
logging.info('Initializing parameters...')
rng = jax.random.PRNGKey(428)
data = next(iter(train_dataset))
state = updater.init(rng, #jnp.broadcast_to(rng, (local_device_count,) + rng.shape), 
                     {k:v[0] for k,v in data.items()},
                     pretrained_params=pretrained_params)
if compute_on_tpu:
    state = jax.device_put_replicated(state, jax.devices())
    update = jax.pmap(updater.update, axis_name='i')
else:
    update = updater.update
    


Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

unused torch weights: roberta.embeddings.token_type_embeddings.weight,
roberta.pooler.dense.weight, roberta.pooler.dense.bias, lm_head.bias,
lm_head.dense.weight, lm_head.dense.bias, lm_head.layer_norm.weight,
lm_head.layer_norm.bias, lm_head.decoder.weight
Starting...
Initializing parameters...


In [ ]:
logging.info('Starting train loop...')
train_iter = iter(train_dataset)
log = Log()
for step  in range(MAX_STEPS): #enumerate(train_dataset):
        data = next(train_iter)
        state = update(state, data)
        # We use JAX runahead to mask data preprocessing and JAX dispatch overheads.
        # Using values from state/metrics too often will block the runahead and can
        # cause these overheads to become more prominent.
        if step % LOG_EVERY == 0 or (step+1)== MAX_STEPS:
            last_log=log.update(state)
            test_loss = eval_net(forward_fn, loss_fn, state, test_dataset)
            last_log['test_loss'] = test_loss #float(state['loss'].mean())
            # state['loss'] = 0*state['loss']
            logging.info(last_log)

# logging.info(last_log)

Starting train loop...
{'step': 1, 'loss': 2.5538339614868164, 'elapsed_time': 114.75881576538086, 'total_time': 114.75881576538086, 'iter_per_sec': 114.75881576538086, 'sec_per_iter': 0.008713927495073269, 'test_loss': 411.80169677734375}


In [ ]:
pred = get_predictions(forward_fn, loss_fn, state=state, test_ds=test_dataset)

In [ ]:
# next(test_dataset.as_numpy_iterator())
#next(iter(test_dataset))

In [ ]:
len(pred), pred[0].shape

In [ ]:
params = hk.data_structures.map(lambda name,module,x: x[0], state['params'])
import pickle
with open('params.pkl','wb') as f:
    pickle.dump(params, f)

In [ ]:
lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=500, 
                                               decay_steps=8500, 
                                               end_value=1e-4)

In [ ]:
import jaxlib, jax
jaxlib.__version__, jax.__version__, hk.__version__

In [ ]:
cnt = np.arange(0,9000,10)
lr = [lr_schedule(jnp.array([i])) for i in cnt]
plt.plot(cnt, lr)

In [ ]:
{'loss': 2.4778389930725098, 'step': 0.0, 'steps_per_sec': 13.779059174634387}
{'loss': 1.3683964014053345, 'step': 500.0, 'steps_per_sec': 2.5767113856201855}
{'loss': 1.6631159782409668, 'step': 1000.0, 'steps_per_sec': 2.57725389686341}
{'loss': 1.3581798076629639, 'step': 1500.0, 'steps_per_sec': 2.5771785561488483}

In [ ]:
jax.core.gc.collect()

In [ ]:
jax.numpy.array([1,1])
#jax.device_get('gpu')
#jax.devices('gpu')
! nvcc --version

In [ ]:
from transformers import TrainingArguments
# TRAINING HYPERPARAMS
effective_batch_size = 32
max_steps = 4000
log_steps = 500
#n_step_examples = 8*500
batch_size = 16
grad_accumulation = effective_batch_size//batch_size
learning_rate = 5e-5
weight_decay = 0.01
warmup_ratio = 0.1
n_epochs = 3
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-{task}",
    evaluation_strategy = "steps",
    eval_steps = log_steps,
    logging_strategy = "steps",
    logging_steps = log_steps,
    save_strategy = "steps",
    save_steps = log_steps,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    #num_train_epochs = n_epochs,
    max_steps = max_steps,
    weight_decay = weight_decay,
    report_to = 'wandb', 
    gradient_accumulation_steps = grad_accumulation,
    warmup_steps = int(1.5*(log_steps)),
    # logging_steps = 100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
)
#training_args

In [ ]:
data['test']=add_rle_word2(data['test'])
data

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=chunk_data["train"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    eval_dataset=chunk_data["test"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=mk_binary_metric(chunk_ds=chunk_data['test'],
                                     orig_ds=data['test'])
    #,
    #                                 min_word_cnt=[0,10,5,10,3,10,6,6]), #.select(range(100))), 
)

In [ ]:
trainer.train()
trainer.save_model(model_path)

In [ ]:
# wandb.init()
import pickle
from feedback2021.postprocess import mk_prediction_transform
for k,v in chunk_data.items():
    predictions = trainer.predict(v.remove_columns(['offset', 'text_id']))
    prediction_transform = mk_binary_prediction_transform(chunk_ds=v, orig_ds=data[k])
    predictions = prediction_transform(predictions[0])
    with open(f'{k}_predictions.pkl','wb') as f:
        pickle.dump(predictions, f)
    wandb.save(f'{k}_predictions.pkl','./','now')

In [ ]:
# ! rm longformer-base-4096-token_classification/ -rf
#!rm distilroberta-base-token_classification -rf
! ls -sh distilroberta-base-1

In [ ]:
wandb.finish()

# From this point on there is no need to gpu!

In [ ]:
prediction_file = wandb.restore('predictions.pkl', run_path='prvi/huggingface/d17yam8m')

In [ ]:
import os

In [ ]:
prediction_file_name = 'test_predictions.pkl' #if os.path.exists('test_predictions.pkl') else prediction_file.name
with open(prediction_file_name, 'rb') as f:
    saved_predictions = pickle.load(f)

In [ ]:
chunk_ds, orig_ds = chunk_data['test'].to_dict(), data['test'].to_dict()

In [ ]:
def filter_prediction(prediction, min_word_cnt):
    return [b for b in prediction if b[2]-b[1]>=min_word_cnt[b[0]]]

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
res = []
for mwc in tqdm(range(0,15)):
#     transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                    orig_ds=orig_ds, min_word_cnt=mwc)
#     predictions = transform_prediction(saved_predictions[0])
    
    scores = np.array([metric.score_example(y_true=y_true, 
                                            y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
    scores = [metric.f1(*score) for score in scores]
    f1 = sum(scores)/len(scores)
    scores = {id2label[i+1]:score for i,score in enumerate(scores)}
    scores['f1'] = f1
    scores['min_word_cnt']=mwc
    res.append(scores)

pd.DataFrame(res).set_index('min_word_cnt')

In [ ]:
mwc =[0, 10, 6, 15, 6, 11, 9, 8]

scores_ = np.array([metric.score_example(y_true=y_true, 
                                        y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
scores = [metric.f1(*score) for score in scores_]
f1 = sum(scores)/len(scores)
scores = {id2label[i+1]:score for i,score in enumerate(scores)}
scores['f1'] = f1
    
# transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                 orig_ds=orig_ds, 
#                                                min_word_cnt=[0, 10, 6, 15, 6, 11, 9, 8])
# predictions = transform_prediction(saved_predictions[0])
# scores = np.array([metric.score_example(y_true=y_true, y_pred=y_pred) 
#               for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]).sum(axis=0)
# scores = [metric.f1(*score) for score in scores]
# f1 = sum(scores)/len(scores)
# scores = {id2label[i+1]:score for i,score in enumerate(scores)}
# scores['f1'] = f1
scores    

In [ ]:
scores_

In [ ]:
predictions = [ sorted(pred, key=lambda x: x[1]) for pred in saved_predictions]

In [ ]:
show_result(i=20, orig_ds=data['test'],predictions=predictions)

In [ ]:

saved_predictions[8],data['test'][8]['rle_token']

In [ ]:
scores = [metric.score_example(y_true=y_true, y_pred=y_pred) for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]

In [ ]:
[metric.f1(*x) for x in np.array(scores).sum(axis=0)]

In [ ]:
idx = np.argsort(np.array(scores)[:,:,1:].sum(axis=(1,2)))[::-1]
idx[:10]